<a href="https://colab.research.google.com/github/prithwis/KKolab/blob/main/KK_C1_SparkSQL_SQLContext_HiveContext.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](https://4.bp.blogspot.com/-gbL5nZDkpFQ/XScFYwoTEII/AAAAAAAAAGY/CcVb_HDLwvs2Brv5T4vSsUcz7O4r2Q79ACK4BGAYYCw/s1600/kk3-header00-beta.png)<br>


<hr>

[Prithwis Mukerjee](http://www.linkedin.com/in/prithwis)<br>

# SPARK SQL
If you wish to run HIVE natively under Hadoop please see this notebook [Hadoop and Hive](https://github.com/prithwis/KKolab/blob/main/KK_B2_Hadoop_and_Hive.ipynb)

# Install Spark

In [1]:
!apt-get update > /dev/null
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#!wget -q http://apache.osuosl.org/spark/spark-2.2.2/spark-2.2.2-bin-hadoop2.7.tgz
#!wget -q http://apache.osuosl.org/spark/spark-2.4.0/spark-2.4.0-bin-hadoop2.7.tgz
#!wget -q http://apache.osuosl.org/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
#!wget -q http://apache.osuosl.org/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
#!wget -q http://apache.osuosl.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop3.2.tgz
!wget -q http://apache.osuosl.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz

#
# if the current version of Spark is not used, there may be errors
# check here for current versions http://apache.osuosl.org/spark
#
#!tar xf spark-2.4.0-bin-hadoop2.7.tgz
#!tar xf spark-2.4.4-bin-hadoop2.7.tgz
#!tar xf spark-2.4.5-bin-hadoop2.7.tgz
#!tar xf spark-3.0.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.2-bin-hadoop3.2.tgz



In [4]:
# !pip install -q findspark  -- findspark is no more required
!pip install -q pyspark

     |████████████████████████████████| 212.4 MB 57 kB/s 
     |████████████████████████████████| 198 kB 32.8 MB/s 


In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
#os.environ["SPARK_HOME"] = "/content/spark-2.4.0-bin-hadoop2.7"
#os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"
#os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"
#os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop3.2"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"

In [5]:
#import findspark
#findspark.init()
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [6]:
sc = spark.sparkContext
#sc

# Load Data

In [7]:
# Data files downloaded from author's G-Drive
!gdown https://drive.google.com/uc?id=1JJH24ZZaiJrEKValD--UtyFcWl7UanwV  # 2% data
!gdown https://drive.google.com/uc?id=1g7mJ0v4fkERW0HWc1eq-SHs_jvQ0N2Oe  # 100% data

Downloading...
From: https://drive.google.com/uc?id=1JJH24ZZaiJrEKValD--UtyFcWl7UanwV
To: /content/eCommerce_02PC_2021.csv
100% 917k/917k [00:00<00:00, 55.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=1g7mJ0v4fkERW0HWc1eq-SHs_jvQ0N2Oe
To: /content/eCommerce_Full_2021.csv
45.6MB [00:00, 97.8MB/s]


In [8]:
#we remove the CRLF character from the end of the row if it exists
#!sed 's/\r//' /content/eCommerce_Full_2021.csv > datafile.csv
!sed 's/\r//' /content/eCommerce_02PC_2021.csv > datafile.csv
#!sed -i -e "1d" datafile.csv               # remove the first line containing headers from the file

In [9]:
data_file = 'datafile.csv'
raw_data = sc.textFile(data_file)
print ("Data Size", raw_data.count())

Data Size 10839


#Spark Data Frame

In [10]:
eCommerce_df = spark.read.csv(data_file,inferSchema=True, header=True)

In [11]:
eCommerce_df.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)



In [12]:
eCommerce_df.select('Quantity','UnitPrice','Country').groupby(eCommerce_df.Country).count().sort('count', ascending = False).show()

+---------------+-----+
|        Country|count|
+---------------+-----+
| United Kingdom| 9964|
|         France|  171|
|        Germany|  169|
|           EIRE|  157|
|        Belgium|   46|
|    Netherlands|   45|
|          Spain|   44|
|    Switzerland|   35|
|       Portugal|   26|
|         Norway|   24|
|      Australia|   18|
|        Finland|   15|
|          Italy|   14|
|        Austria|   13|
|          Japan|   12|
|    Unspecified|   11|
|         Cyprus|   10|
|         Sweden|    7|
|        Denmark|    6|
|Channel Islands|    6|
+---------------+-----+
only showing top 20 rows



#Spark SQL
What is the difference between SQLContext and HiveContext? See [here](https://intellipaat.com/community/7599/what-is-the-difference-between-apache-spark-sqlcontext-vs-hivecontext#:~:text=HiveContext%20is%20a%20super%20set,read%20data%20from%20Hive%20tables.&text=The%20more%20basic%20SQLContext%20provides,does%20not%20depend%20on%20Hive.), or [here](https://stackoverflow.com/questions/33666545/what-is-the-difference-between-apache-spark-sqlcontext-vs-hivecontext)

## Remove comma from data of CSV file

In [13]:
#we remove the CRLF character from the end of the row if it exists
#!sed 's/\r//' /content/eCommerce_Full_2021.csv > datafile.csv
!sed 's/\r//g' /content/eCommerce_02PC_2021.csv > datafile.csv
# remove the first line containing headers from the file
!sed -i -e "1d" datafile.csv               

The data contains commas which causes havoc during the load process

In [14]:
!grep 'TROPICAL BEACH' datafile.csv

548196,82011A,"BATHROOM SCALES, TROPICAL BEACH",1,3/29/2011 15:44,3.75,16011.0,United Kingdom


In [15]:
!grep 'COFFEE,' datafile.csv

563020,21216,"SET 3 RETROSPOT TEA,COFFEE,SUGAR",4,8/11/2011 12:58,4.95,12428.0,Finland
555744,85159A,"BLACK TEA,COFFEE,SUGAR JARS",24,6/7/2011 10:44,1.95,18072.0,United Kingdom
539581,21216,"SET 3 RETROSPOT TEA,COFFEE,SUGAR",4,12/20/2010 12:51,4.95,16143.0,United Kingdom
545715,21216,"SET 3 RETROSPOT TEA,COFFEE,SUGAR",2,3/7/2011 10:13,10.79,,United Kingdom
561678,21216,"SET 3 RETROSPOT TEA,COFFEE,SUGAR",1,7/28/2011 18:33,4.95,17931.0,United Kingdom
566041,21216,"SET 3 RETROSPOT TEA,COFFEE,SUGAR",8,9/8/2011 13:48,4.95,13814.0,Germany
566073,21216,"SET 3 RETROSPOT TEA,COFFEE,SUGAR",2,9/8/2011 19:58,4.95,14562.0,United Kingdom
568712,21216,"SET 3 RETROSPOT TEA,COFFEE,SUGAR",1,9/28/2011 15:50,4.95,17841.0,United Kingdom
555327,21216,"SET 3 RETROSPOT TEA,COFFEE,SUGAR",2,6/2/2011 11:03,10.79,,United Kingdom
540093,21216,"SET 3 RETROSPOT TEA,COFFEE,SUGAR",1,1/4/2011 15:24,4.95,16725.0,United Kingdom
558538,21216,"SET 3 RETROSPOT TEA,COFFEE,SUGAR",2,6/30/2011 12:12,4.95,15005.0,United Kingdom


In [16]:
!head datafile.csv

C544414,22960,JAM MAKING SET WITH JARS,-2,2/18/2011 14:54,3.75,13408.0,United Kingdom
555276,48111,DOORMAT 3 SMILEY CATS,1,6/1/2011 17:28,15.79,,United Kingdom
575656,22952,60 CAKE CASES VINTAGE CHRISTMAS,48,11/10/2011 14:29,0.55,13319.0,United Kingdom
571636,20674,GREEN POLKADOT BOWL,16,10/18/2011 11:41,1.25,13509.0,United Kingdom
576657,22556,PLASTERS IN TIN CIRCUS PARADE ,12,11/16/2011 11:03,1.65,12720.0,Germany
569823,23298,SPOTTY BUNTING,1,10/6/2011 12:15,4.95,16895.0,United Kingdom
570185,21090,WET/MOULDY,-192,10/7/2011 14:56,0.0,,United Kingdom
574943,17091J,VANILLA INCENSE IN TIN,36,11/8/2011 7:52,0.38,13026.0,United Kingdom
561902,22909,SET OF 20 VINTAGE CHRISTMAS NAPKINS,96,7/31/2011 15:50,0.85,17404.0,Sweden
550499,22138,BAKING SET 9 PIECE RETROSPOT ,1,4/18/2011 15:16,10.79,,United Kingdom


commas getting removed here

In [18]:
import pandas as pd

In [19]:
p_df = pd.read_csv('datafile.csv',header=None)

In [20]:
p_df2 = p_df.replace(',', ' ', regex=True)

In [21]:
p_df2.to_csv('datafile2.csv',index=False,sep=',',header=False)

In [22]:
!grep 'TROPICAL BEACH' datafile2.csv

548196,82011A,BATHROOM SCALES  TROPICAL BEACH,1,3/29/2011 15:44,3.75,16011.0,United Kingdom


In [ ]:
#!grep 'COFFEE' datafile2.csv

##SQL Context

In [23]:
from pyspark.sql import SQLContext
from pyspark.sql import Row

sqlContext = SQLContext(sc)

In [24]:
data_file = "datafile2.csv"
raw_data = sc.textFile(data_file)
csv_data = raw_data.map(lambda l: l.split(","))

In [25]:
csv_data.take(2)

[['C544414',
  '22960',
  'JAM MAKING SET WITH JARS',
  '-2',
  '2/18/2011 14:54',
  '3.75',
  '13408.0',
  'United Kingdom'],
 ['555276',
  '48111',
  'DOORMAT 3 SMILEY CATS',
  '1',
  '6/1/2011 17:28',
  '15.79',
  '',
  'United Kingdom']]

In [26]:
row_data = csv_data.map(lambda p: Row(
    InvoiceNo=p[0], 
    StockCode=p[1],
    Description=p[2],
    Quantity=int(p[3]),
    InvoiceDate=p[4],
    UnitPrice=float(p[5]),
    CustomerID=p[6],
    Country=p[7]
    )
)

In [27]:
row_data.take(2)

[Row(InvoiceNo='C544414', StockCode='22960', Description='JAM MAKING SET WITH JARS', Quantity=-2, InvoiceDate='2/18/2011 14:54', UnitPrice=3.75, CustomerID='13408.0', Country='United Kingdom'),
 Row(InvoiceNo='555276', StockCode='48111', Description='DOORMAT 3 SMILEY CATS', Quantity=1, InvoiceDate='6/1/2011 17:28', UnitPrice=15.79, CustomerID='', Country='United Kingdom')]

In [28]:
eCommerce2_df = sqlContext.createDataFrame(row_data)
eCommerce2_df.registerTempTable("eCommerce")

In [29]:
Countries = sqlContext.sql("SELECT * FROM eCommerce limit 20")
Countries.show()

+---------+---------+--------------------+--------+----------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|     InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+----------------+---------+----------+--------------+
|  C544414|    22960|JAM MAKING SET WI...|      -2| 2/18/2011 14:54|     3.75|   13408.0|United Kingdom|
|   555276|    48111|DOORMAT 3 SMILEY ...|       1|  6/1/2011 17:28|    15.79|          |United Kingdom|
|   575656|    22952|60 CAKE CASES VIN...|      48|11/10/2011 14:29|     0.55|   13319.0|United Kingdom|
|   571636|    20674| GREEN POLKADOT BOWL|      16|10/18/2011 11:41|     1.25|   13509.0|United Kingdom|
|   576657|    22556|PLASTERS IN TIN C...|      12|11/16/2011 11:03|     1.65|   12720.0|       Germany|
|   569823|    23298|      SPOTTY BUNTING|       1| 10/6/2011 12:15|     4.95|   16895.0|United Kingdom|
|   570185|    21090|          WET/MOULDY|    -192| 10/

In [30]:
Countries = sqlContext.sql("SELECT sum(Quantity), sum(UnitPrice), Country from eCommerce group by Country order by sum(Quantity) desc")
Countries.show()

+-------------+------------------+---------------+
|sum(Quantity)|    sum(UnitPrice)|        Country|
+-------------+------------------+---------------+
|        88067| 44191.70000000067| United Kingdom|
|         3762|             86.12|    Netherlands|
|         2725| 525.6800000000001|         France|
|         2266| 680.6699999999995|           EIRE|
|         1758| 567.3799999999997|        Germany|
|         1567| 54.92999999999999|      Australia|
|          735|28.759999999999998|          Japan|
|          520|179.85000000000005|        Belgium|
|          405|143.37000000000003|    Switzerland|
|          385|             49.47|         Sweden|
|          362|            135.62|          Spain|
|          319|            106.17|         Norway|
|          181|            471.33|       Portugal|
|          179|             75.02|        Finland|
|          144|              40.9|      Hong Kong|
|          127|           2065.98|      Singapore|
|          118|             12.

## HIVE context
this part adapted from [this notebook](https://colab.research.google.com/github/jmbanda/BigDataProgramming_2019/blob/master/Chapter_5_Loading_and_Saving_Data_in_Spark.ipynb#scrollTo=lOcCViaN4cwC)

In [31]:
from pyspark.sql import Row
from pyspark.sql import HiveContext
sqlContext = HiveContext(sc)

In [32]:
data_file = "datafile2.csv"
raw_data = sc.textFile(data_file)
csv_data = raw_data.map(lambda l: l.split(","))

In [33]:
row_data = csv_data.map(lambda p: Row(
    InvoiceNo=p[0], 
    StockCode=p[1],
    Description=p[2],
    Quantity=int(p[3]),
    InvoiceDate=p[4],
    UnitPrice=float(p[5]),
    CustomerID=p[6],
    Country=p[7]
    )
)

In [34]:
ec_schema = sqlContext.createDataFrame(row_data)
# Register it as a temp table
sqlContext.registerDataFrameAsTable(ec_schema, "ec_HiveTable")
sqlContext.sql("show tables").show()

+--------+------------+-----------+
|database|   tableName|isTemporary|
+--------+------------+-----------+
|        |ec_hivetable|       true|
|        |   ecommerce|       true|
+--------+------------+-----------+



In [35]:
Countries = sqlContext.sql("SELECT * FROM ec_HiveTable limit 20")
Countries.show()

+---------+---------+--------------------+--------+----------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|     InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+----------------+---------+----------+--------------+
|  C544414|    22960|JAM MAKING SET WI...|      -2| 2/18/2011 14:54|     3.75|   13408.0|United Kingdom|
|   555276|    48111|DOORMAT 3 SMILEY ...|       1|  6/1/2011 17:28|    15.79|          |United Kingdom|
|   575656|    22952|60 CAKE CASES VIN...|      48|11/10/2011 14:29|     0.55|   13319.0|United Kingdom|
|   571636|    20674| GREEN POLKADOT BOWL|      16|10/18/2011 11:41|     1.25|   13509.0|United Kingdom|
|   576657|    22556|PLASTERS IN TIN C...|      12|11/16/2011 11:03|     1.65|   12720.0|       Germany|
|   569823|    23298|      SPOTTY BUNTING|       1| 10/6/2011 12:15|     4.95|   16895.0|United Kingdom|
|   570185|    21090|          WET/MOULDY|    -192| 10/

In [36]:
Countries = sqlContext.sql("SELECT sum(Quantity), sum(UnitPrice), Country from eCommerce group by Country order by sum(Quantity) desc")
Countries.show()

+-------------+------------------+---------------+
|sum(Quantity)|    sum(UnitPrice)|        Country|
+-------------+------------------+---------------+
|        88067| 44191.70000000067| United Kingdom|
|         3762|             86.12|    Netherlands|
|         2725| 525.6800000000001|         France|
|         2266| 680.6699999999995|           EIRE|
|         1758| 567.3799999999997|        Germany|
|         1567| 54.92999999999999|      Australia|
|          735|28.759999999999998|          Japan|
|          520|179.85000000000005|        Belgium|
|          405|143.37000000000003|    Switzerland|
|          385|             49.47|         Sweden|
|          362|            135.62|          Spain|
|          319|            106.17|         Norway|
|          181|            471.33|       Portugal|
|          179|             75.02|        Finland|
|          144|              40.9|      Hong Kong|
|          127|           2065.98|      Singapore|
|          118|             12.

#Chronobooks <br>
![alt text](https://1.bp.blogspot.com/-lTiYBkU2qbU/X1er__fvnkI/AAAAAAAAjtE/GhDR3OEGJr4NG43fZPodrQD5kbxtnKebgCLcBGAsYHQ/s600/Footer2020-600x200.png)<hr>
Chronotantra and Chronoyantra are two science fiction novels that explore the collapse of human civilisation on Earth and then its rebirth and reincarnation both on Earth as well as on the distant worlds of Mars, Titan and Enceladus. But is it the human civilisation that is being reborn? Or is it some other sentience that is revealing itself. 
If you have an interest in AI and found this material useful, you may consider buying these novels, in paperback or kindle, from [http://bit.ly/chronobooks](http://bit.ly/chronobooks)